In [1]:
from tkinter import *
from tkinter import filedialog
from PIL import ImageTk, Image
import cv2
import numpy as np

In [2]:
def loadCascadeFiles():
    global haardata
    global mouth_cascade
    global nose_cascade
    haardata = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    mouth_cascade = cv2.CascadeClassifier('haarcascade_mcs_mouth.xml')
    nose_cascade = cv2.CascadeClassifier('haarcascade_mcs_nose.xml')

## IMAGE

In [3]:
def openImageWindow():
    # Setting Window Close
    allWindowClose = "Image"
    
    # Disabling all Button
    image['state'] = 'disabled'
    video['state'] = 'disabled'
    live['state'] = 'disabled'
    
    # File Dialog
    root.filename = filedialog.askopenfilename(initialdir = '/Users/SURAJ/Project',
                                               title = 'OPEN', 
                                               filetypes = (('png files', '*.png'), ('allfiles', '*.*')))
    
    # Reading image to check if it is not None
    checkImg = cv2.imread(root.filename)
    
    # Checking if img is empty or not
    if np.all(checkImg == None):
        image['state'] = 'normal'
        video['state'] = 'normal'
        live['state'] = 'normal'
        return
    
    # Creating Image Window
    global imageWindow
    imageWindow = Toplevel(root)
    imageWindow.title('Image Window')
    imageWindow.resizable(False, False)
    imageWindow.iconbitmap('winlogo.ico')
    imageWindow.geometry(str(int(checkImg.shape[1]/2)) + "x" + str(int(checkImg.shape[0]/2)))
    imageWindow.protocol("WM_DELETE_WINDOW", onImageWindowClose)
    
    # Adding Label
    global imageLabel
    imageLabel = Label(imageWindow)
    imageLabel.pack()
    
    #openImageControlPanel()
    openControlPanel(imageWindow)
    
    processImage(0)

In [4]:
def processImage(var):
    #print('SLIDER')
    
    global img
    img = cv2.imread(root.filename)
    
    # Getting width and length of image
    height, width, layers = img.shape
    new_h = int(height / 2)
    new_w = int(width / 2)
    
    # Resizing the image
    img = cv2.resize(img, (new_w, new_h))
    
    # Converting the image to gray scale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Detecting Faces        1.1
    faces = haardata.detectMultiScale(gray, scaleFactor = faceScaleFactor.get(), minNeighbors = faceMinNeigh.get())
    
    for x, y, w, h in faces:
        x = x + 1
        y = y + 1
        w = w - 1
        h = h - 1

        # Drawing rectangle around face
        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 255), 3)
        
        # Croping the face from frame, converting it to Gray and Displaying it
        face = img[y:y + h, x:x + w, :]
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        #cv2.imshow('FACE', face)


        # Detecting mouth on the face
        mouths = mouth_cascade.detectMultiScale(image = face, scaleFactor = mouthScaleFactor.get(), 
                                                minNeighbors = mouthMinNeigh.get())

        for (x1,y1,w1,h1) in mouths:
            y1 = int(y1 - 0.15 * h1)

            # Croping mouth and displaying it
            mouth = face[y1:y1 + h1, x1:x1 + w1]
            #cv2.imshow('MOUTH', mouth)

            # Drawing rectangle around the mouth
            cv2.rectangle(img, (x + x1,y + y1), (x + x1 + w1,y + y1 + h1), (0,255,0), 3)

            # Breaking it
            break


        # Detecting Nose on the face
        noses = nose_cascade.detectMultiScale(image = face, scaleFactor = noseScaleFactor.get(), 
                                              minNeighbors = noseMinNeigh.get())

        for (x1,y1,w1,h1) in noses:
            y1 = int(y1 - 0.15 * h1)

            # Croping mouth and displaying it
            nose = face[y1:y1 + h1, x1:x1 + w1]
            #cv2.imshow('NOSE', nose)

            # Drawing rectangle around the mouth
            cv2.rectangle(img, (x + x1,y + y1), (x + x1 + w1,y + y1 + h1), (0,255,0), 3)

            # Breaking it
            break


        # Checking if we can identify mouth in the face
        if len(mouths) > 0 and len(noses) > 0:
            cv2.putText(img, 'No Mask', (x, y), font, 1, (0, 255, 255), 2)
        elif len(noses) > 0 and len(mouths) == 0:
            cv2.putText(img, 'Half Mask', (x, y), font, 1, (0, 255, 255), 2)
        else:
            cv2.putText(img, 'Mask', (x, y), font, 1, (0, 255, 255), 2)

        
        break
        
        
    height, width = img.shape[:2]
    ppm_header = f'P6 {width} {height} 255 '.encode()
    data = ppm_header + cv2.cvtColor(img, cv2.COLOR_BGR2RGB).tobytes()
    imageFrame = PhotoImage(width=width, height=height, data=data, format='PPM')
    
    imageLabel.configure(image = imageFrame)
    imageLabel.photo = imageFrame

## VIDEO

In [5]:
def openVideoWindow():
    # Setting Window Close
    allWindowClose = "Video"
    
    # Disabling all Button
    image['state'] = 'disabled'
    video['state'] = 'disabled'
    live['state'] = 'disabled'
    
    # File Dialog
    root.filename = filedialog.askopenfilename(initialdir = '/Users/SURAJ/Project',
                                               title = 'OPEN', 
                                               filetypes = (('mp4 files', '*.mp4'), ('allfiles', '*.*')))
    
    global vCapture
    vCapture = cv2.VideoCapture(root.filename)
    flag, img = vCapture.read()
    
    # Checking if img is empty or not
    if np.all(img == None):
        image['state'] = 'normal'
        video['state'] = 'normal'
        live['state'] = 'normal'
        return
    
    # Creating Live Window
    global videoWindow
    videoWindow = Toplevel(root)
    videoWindow.title('Video Window')
    videoWindow.resizable(False, False)
    videoWindow.iconbitmap('winlogo.ico')
    #videoWindow.geometry("640x480")
    videoWindow.protocol("WM_DELETE_WINDOW", onVideoWindowClose)
    
    # Label to display frame
    global videoImageLabel
    videoImageLabel = Label(videoWindow)
    
    processVideo()

In [6]:
def processVideo():
    # Control Panel for Video Window
    openControlPanel(videoWindow)
    
    # Video Window Loop
    while True:
        flag, img = vCapture.read()
        
        if flag:
            height, width, layers = img.shape
            new_h = int(height / 2)
            new_w = int(width / 2)
            img = cv2.resize(img, (new_w, new_h))
            
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = haardata.detectMultiScale(gray, scaleFactor = faceScaleFactor.get(),
                                             minNeighbors = faceMinNeigh.get())

            for x, y, w, h in faces:
                x = x + 1
                y = y + 1
                w = w - 1
                h = h - 1

                # Drawing rectangle around face
                cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 255), 3)

                # Croping the face from frame, converting it to Gray and Displaying it
                face = img[y:y + h, x:x + w, :]
                face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
                #cv2.imshow('FACE', face)


                # Detecting mouth on the face
                mouths = mouth_cascade.detectMultiScale(image = face, scaleFactor = mouthScaleFactor.get(), 
                                                        minNeighbors = mouthMinNeigh.get())

                for (x1,y1,w1,h1) in mouths:
                    y1 = int(y1 - 0.15 * h1)

                    # Croping mouth and displaying it
                    mouth = face[y1:y1 + h1, x1:x1 + w1]
                    #cv2.imshow('MOUTH', mouth)

                    # Drawing rectangle around the mouth
                    cv2.rectangle(img, (x + x1,y + y1), (x + x1 + w1,y + y1 + h1), (0,255,0), 3)

                    # Breaking it
                    break


                # Detecting Nose on the face
                noses = nose_cascade.detectMultiScale(image = face, scaleFactor = noseScaleFactor.get(), 
                                                      minNeighbors = noseMinNeigh.get())

                for (x1,y1,w1,h1) in noses:
                    y1 = int(y1 - 0.15 * h1)

                    # Croping mouth and displaying it
                    nose = face[y1:y1 + h1, x1:x1 + w1]
                    #cv2.imshow('NOSE', nose)

                    # Drawing rectangle around the mouth
                    cv2.rectangle(img, (x + x1,y + y1), (x + x1 + w1,y + y1 + h1), (0,255,0), 3)

                    # Breaking it
                    break


                # Checking if we can identify mouth in the face
                if len(mouths) > 0 and len(noses) > 0:
                    cv2.putText(img, 'No Mask', (x, y), font, 1, (0, 255, 255), 2)
                elif len(noses) > 0 and len(mouths) == 0:
                    cv2.putText(img, 'Half Mask', (x, y), font, 1, (0, 255, 255), 2)
                else:
                    cv2.putText(img, 'Mask', (x, y), font, 1, (0, 255, 255), 2)

                break
        
    
        # Checking if window is closed
        if np.all(img == None):
            break
    
        # Displaying the image on the screen
        height, width = img.shape[:2]
        ppm_header = f'P6 {width} {height} 255 '.encode()
        data = ppm_header + cv2.cvtColor(img, cv2.COLOR_BGR2RGB).tobytes()

        imageFrame = PhotoImage(width = width, height = height, data = data, format = 'PPM')
        
        videoImageLabel['image'] = imageFrame
        videoImageLabel.pack()
        
        videoImageLabel.update()
        
    onVideoWindowClose()

## LIVE

In [7]:
def openLiveWindow():
    # Setting Window Close
    allWindowClose = "Live"
    
    # Disabling all Button
    image['state'] = 'disabled'
    video['state'] = 'disabled'
    live['state'] = 'disabled'
    
    global lcapture
    lcapture = cv2.VideoCapture(0)
    flag, img = lcapture.read()
    
    # Creating Live Window
    global liveWindow
    liveWindow = Toplevel(root)
    liveWindow.title('Live Window')
    liveWindow.resizable(False, False)
    liveWindow.iconbitmap('winlogo.ico')
    liveWindow.geometry("640x480")
    liveWindow.protocol("WM_DELETE_WINDOW", onLiveWindowClose)
    
    # Label to display frame
    global liveImageLabel
    liveImageLabel = Label(liveWindow)
    
    processLive()

In [8]:
def processLive():
    # Control Panel for Live Window
    openControlPanel(liveWindow)
    
    # Live Window Loop
    while True:
        flag, img = lcapture.read()
        
        if flag:
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = haardata.detectMultiScale(gray, scaleFactor = faceScaleFactor.get(),
                                             minNeighbors = faceMinNeigh.get())

            for x, y, w, h in faces:
                x = x + 1
                y = y + 1
                w = w - 1
                h = h - 1

                # Drawing rectangle around face
                cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 255), 3)

                # Croping the face from frame, converting it to Gray and Displaying it
                face = img[y:y + h, x:x + w, :]
                face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
                #cv2.imshow('FACE', face)


                # Detecting mouth on the face
                mouths = mouth_cascade.detectMultiScale(image = face, scaleFactor = mouthScaleFactor.get(), 
                                                        minNeighbors = mouthMinNeigh.get())

                for (x1,y1,w1,h1) in mouths:
                    y1 = int(y1 - 0.15 * h1)

                    # Croping mouth and displaying it
                    mouth = face[y1:y1 + h1, x1:x1 + w1]
                    #cv2.imshow('MOUTH', mouth)

                    # Drawing rectangle around the mouth
                    cv2.rectangle(img, (x + x1,y + y1), (x + x1 + w1,y + y1 + h1), (0,255,0), 3)

                    # Breaking it
                    break


                # Detecting Nose on the face
                noses = nose_cascade.detectMultiScale(image = face, scaleFactor = noseScaleFactor.get(), 
                                                      minNeighbors = noseMinNeigh.get())

                for (x1,y1,w1,h1) in noses:
                    y1 = int(y1 - 0.15 * h1)

                    # Croping mouth and displaying it
                    nose = face[y1:y1 + h1, x1:x1 + w1]
                    #cv2.imshow('NOSE', nose)

                    # Drawing rectangle around the mouth
                    cv2.rectangle(img, (x + x1,y + y1), (x + x1 + w1,y + y1 + h1), (0,255,0), 3)

                    # Breaking it
                    break


                # Checking if we can identify mouth in the face
                if len(mouths) > 0 and len(noses) > 0:
                    cv2.putText(img, 'No Mask', (x, y), font, 1, (0, 255, 255), 2)
                elif len(noses) > 0 and len(mouths) == 0:
                    cv2.putText(img, 'Half Mask', (x, y), font, 1, (0, 255, 255), 2)
                else:
                    cv2.putText(img, 'Mask', (x, y), font, 1, (0, 255, 255), 2)

                break
        
    
        # Checking if window is closed
        if liveImageLabel.winfo_exists() != 1:
            break
    
        # Displaying the image on the screen
        height, width = img.shape[:2]
        ppm_header = f'P6 {width} {height} 255 '.encode()
        data = ppm_header + cv2.cvtColor(img, cv2.COLOR_BGR2RGB).tobytes()

        imageFrame = PhotoImage(width = width, height = height, data = data, format = 'PPM')
        
        liveImageLabel['image'] = imageFrame
        liveImageLabel.pack()
        
        liveImageLabel.update()

## Control Panel

In [9]:
def openControlPanel(window):
    # Checking which window it is
    if window.title() == "Image Window":
        commandParameter = processImage
    else:
        commandParameter = None
    
    # Creating Control Panel Window
    controlPanel = Toplevel(window)
    controlPanel.title('Control Panel')
    controlPanel.geometry("440x500")
    controlPanel.iconbitmap('control.ico')
    controlPanel.resizable(False, False)
    
    # Creating Frame for Face
    faceFrame = LabelFrame(controlPanel, text = "Face", padx = 10, pady = 10)
    faceFrame.place(x = 20, y = 10)

    # Face Scale Factor
    global faceScaleFactor
    faceScaleFactor = Scale(faceFrame, label = 'Scale Factor', from_ = 1.1, to = 3.2,
                          orient = HORIZONTAL, resolution = 0.3, command = commandParameter)
    faceScaleFactor.set(1.1)
    faceScaleFactor.pack()

    # Face Min Neighbors
    global faceMinNeigh
    faceMinNeigh = Scale(faceFrame, label = 'Min Neighbors', from_ = 3, to = 8,
                          orient = HORIZONTAL, resolution = 1, command = commandParameter)
    faceMinNeigh.set(6)
    faceMinNeigh.pack()



    # Creating Frame for Nose
    noseFrame = LabelFrame(controlPanel, text = "Nose", padx = 10, pady = 10)
    noseFrame.place(x = 20, y = 170)

    # Nose Scale Factor
    global noseScaleFactor
    noseScaleFactor = Scale(noseFrame, label = 'Scale Factor', from_ = 1.1, to = 3.5,
                          orient = HORIZONTAL, resolution = 0.2, command = commandParameter)
    noseScaleFactor.set(1.5)
    noseScaleFactor.pack()

    # Nose Min Neighbors
    global noseMinNeigh
    noseMinNeigh = Scale(noseFrame, label = 'Min Neighbors', from_ = 4, to = 9,
                          orient = HORIZONTAL, resolution = 1, command = commandParameter)
    noseMinNeigh.set(7)
    noseMinNeigh.pack()


    # Creating Frame for Mouth
    mouthFrame = LabelFrame(controlPanel, text = "Mouth", padx = 10, pady = 10)
    mouthFrame.place(x = 20, y = 330)

    # Mouth Scale Factor
    global mouthScaleFactor
    mouthScaleFactor = Scale(mouthFrame, label = 'Scale Factor', from_ = 1.2, to = 3.6,
                          orient = HORIZONTAL, resolution = 0.2, command = commandParameter)
    mouthScaleFactor.set(1.6)
    mouthScaleFactor.pack()

    # Mouth Min Neighbors
    global mouthMinNeigh
    mouthMinNeigh = Scale(mouthFrame, label = 'Min Neighbors', from_ = 9, to = 17,
                          orient = HORIZONTAL, resolution = 1, command = commandParameter)
    mouthMinNeigh.set(15)
    mouthMinNeigh.pack()
    
    
    
    # Creating Frame for INFO
    info = LabelFrame(controlPanel, text = "INFO", padx = 10, pady = 10)
    info.place(x = 160, y = 10)

    # Text inside the Labels
    scaleInfoText = "----- Scale Factor -----\nParameter specifying how much the\nimage size is reduced at each image scale.\nModel has a fixed size defined during\ntraining, which is visible in the XML.\nThis means that this size of the face is\ndetected in the image if present.\nHowever, by rescaling the input image,\nyou can resize a larger face to a smaller one,\nmaking it detectable by the algorithm.\n1.05 is a good possible value.\nYou may increase it to as much as 1.4 for\nfaster detection, with the risk of missing\nsome faces altogether."
    minInfoText = "----- MinNeighbors -----\nParameter specifying how many \nneighbors each candidate rectangle\nshould have to retain it.\nThis parameter will affect the quality\nof the detected faces.\nHigher value results in fewer detections\nbut with higher quality.\n3~6 is a good value for it."

    # Scale Factor Info Label
    infoScale = Label(info, text = scaleInfoText)
    infoScale.pack()

    # Min Neighbors Info Label
    infoMin = Label(info, text = minInfoText)
    infoMin.pack()

## Close Functions

In [10]:
def onImageWindowClose():
    # Enabling all Buttons
    image['state'] = 'normal'
    video['state'] = 'normal'
    live['state'] = 'normal'
    
    # Destroying the window
    imageWindow.destroy()

In [11]:
def onVideoWindowClose():
    # Enabling all Buttons
    image['state'] = 'normal'
    video['state'] = 'normal'
    live['state'] = 'normal'
    
    # Destroying the window
    videoWindow.destroy()
    
    # Releasing the resource
    vCapture.release()

In [12]:
def onLiveWindowClose():
    # Enabling all Buttons
    image['state'] = 'normal'
    video['state'] = 'normal'
    live['state'] = 'normal'
    
    # Releasing the resource
    lcapture.release()
    
    # Destroying the window
    liveWindow.destroy()

In [13]:
def onMainWindowClose():
    if allWindowClose == 'Video':
        # Destroying the window
        videoWindow.destroy()

        # Releasing the resource
        vCapture.release()
    elif allWindowClose == 'Live':
        # Releasing the resource
        lcapture.release()

        # Destroying the window
        liveWindow.destroy()
    elif allWindowClose == 'Image':
        # Destroying the window
        imageWindow.destroy()
    
    root.destroy()

## Main

In [ ]:
root = Tk()
root.iconbitmap(r'winlogo.ico')
root.title('Mask Detection')
root.geometry("550x250")
root.resizable(False, False)
root.configure(bg = 'gray')
# TODOs
root.protocol("WM_DELETE_WINDOW", onMainWindowClose)

global allWindowClose
allWindowClose = ""

# Mask Detection Image
logo = ImageTk.PhotoImage(Image.open('mask.png'))
logolabel = Label(root, image = logo)
logolabel.place(x = 10, y = 10)

# Mask Label
namemask = Label(root, text = 'MASK', font = ('Consolas', 25, 'bold'), bg = 'gray', fg = 'light blue')
namemask.place(x = 270, y = 20)

# Detection Label
namedet = Label(root, text = 'DETECTION', font = ('Consolas', 25, 'bold'), bg = 'gray', fg = 'light blue')
namedet.place(x = 270, y = 60)

# Image Button
image = Button(root, text = 'IMAGE', padx = 10, pady = 10, command = openImageWindow)
image.place(x = 270, y = 130)

# Video Button
video = Button(root, text = 'VIDEO', padx = 10, pady = 10, command = openVideoWindow)
video.place(x = 350, y = 130)

# Live Button
live = Button(root, text = 'LIVE', padx = 10, pady = 10, command = openLiveWindow)
live.place(x = 430, y = 130)

# Laoding all cascade files
loadCascadeFiles()

# Checking if Cascade Files are not empty
if haardata.empty() or nose_cascade.empty() or mouth_cascade.empty():
    raise IOError('Unable to load Cascade file')

# Setting font
font = cv2.FONT_HERSHEY_SIMPLEX
    
# while True:
#     root.update()
root.mainloop()